# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [74]:
# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
import pickle
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("disaster_reports", con=engine)
X = df["message"]
Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)

In [3]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text, lemmatizer=WordNetLemmatizer()):
    
    # Detecte URLs
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
        
    # Normalize and tokenize
    tokens = nltk.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))
    
    # Remove stopwords
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    
    # Lemmatize
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [26]:
pipeline =  Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.2)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def Report(model, X_test, y_test):
    
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

Report(pipeline, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.60      0.44      0.51      1163
          1       0.85      0.91      0.88      4041
          2       0.21      0.40      0.28        40

avg / total       0.79      0.80      0.79      5244

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4347
          1       0.81      0.42      0.55       897

avg / total       0.88      0.88      0.87      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5221
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.75      0.84      0.79      3082
          1       0.73      0.61      0.66      2162

avg / total       0.74      0.74      0.74      5244

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [37]:
parameters =  { 'clf__estimator__criterion':['entropy'],
                'clf__estimator__n_estimators': [20, 50], 
                'clf__estimator__max_depth': [2, 5],
                'tfidf__use_idf': (True, False),
                'clf__estimator__min_samples_leaf':[1,5]
            }

cv = GridSearchCV(pipeline, param_grid = parameters,verbose=10)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__criterion': ['entropy'], 'clf__estimator__n_estimators': [20, 50], 'clf__estimator__max_depth': [2, 5], 'tfidf__use_idf': (True, False), 'clf__estimator__min_samples_leaf': [1, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [38]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=20, tfidf__use_idf=True 
[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=20, tfidf__use_idf=True, score=0.02531826634244028, total= 1.5min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=20, tfidf__use_idf=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=20, tfidf__use_idf=True, score=0.006722929480760979, total= 1.6min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=20, tfidf__use_idf=True 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.1min remaining:    0.0s


[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=20, tfidf__use_idf=True, score=0.06652360515021459, total= 1.6min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=20, tfidf__use_idf=False 


...

[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.09227467811158799, total= 1.6min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=50, tfidf__use_idf=False 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 23.4min remaining:    0.0s


[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.069374910599342, total= 1.6min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.09183235588613932, total= 1.6min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.09227467811158799, total= 1.6min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_depth=2, clf__es

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 126.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__criterion': ['entropy'], 'clf__estimator__n_estimators': [20, 50], 'clf__estimator__max_depth': [2, 5], 'tfidf__use_idf': (True, False), 'clf__estimator__min_samples_leaf': [1, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [39]:
Report(cv, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.35      0.78      0.48      1163
          1       0.90      0.58      0.71      4041
          2       0.18      0.23      0.20        40

avg / total       0.77      0.62      0.65      5244

request
             precision    recall  f1-score   support

          0       0.93      0.88      0.90      4347
          1       0.53      0.68      0.60       897

avg / total       0.86      0.84      0.85      5244

offer
             precision    recall  f1-score   support

          0       1.00      0.99      0.99      5221
          1       0.04      0.09      0.06        23

avg / total       0.99      0.99      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.77      0.81      0.79      3082
          1       0.70      0.65      0.67      2162

avg / total       0.74      0.74      0.74      5244

medical_help
             precision    recall  f1-sco

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [45]:

new_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [46]:
new_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [47]:
Report(new_pipeline, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.61      0.09      0.15      1163
          1       0.78      0.98      0.87      4041
          2       0.53      0.23      0.32        40

avg / total       0.74      0.78      0.71      5244

request
             precision    recall  f1-score   support

          0       0.90      0.96      0.93      4347
          1       0.74      0.51      0.60       897

avg / total       0.88      0.89      0.88      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5221
          1       0.00      0.00      0.00        23

avg / total       0.99      0.99      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.75      0.88      0.81      3082
          1       0.77      0.59      0.67      2162

avg / total       0.76      0.76      0.75      5244

medical_help
             precision    recall  f1-sco

In [57]:
new_pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(
        AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))))
    
])

In [58]:
new_pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...er='best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [59]:
Report(new_pipeline2, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.35      0.79      0.48      1163
          1       0.92      0.54      0.68      4041
          2       0.09      0.60      0.16        40

avg / total       0.79      0.59      0.63      5244

request
             precision    recall  f1-score   support

          0       0.94      0.85      0.89      4347
          1       0.50      0.74      0.60       897

avg / total       0.87      0.83      0.84      5244

offer
             precision    recall  f1-score   support

          0       1.00      0.93      0.96      5221
          1       0.02      0.35      0.04        23

avg / total       0.99      0.93      0.96      5244

aid_related
             precision    recall  f1-score   support

          0       0.79      0.81      0.80      3082
          1       0.72      0.70      0.71      2162

avg / total       0.76      0.77      0.76      5244

medical_help
             precision    recall  f1-sco

In [60]:
new_pipeline3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD()),
    ('clf', MultiOutputClassifier(
        AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))))
    
])

In [61]:
new_pipeline3.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...er='best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [62]:
Report(new_pipeline3, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.34      0.51      0.41      1163
          1       0.85      0.67      0.75      4041
          2       0.05      0.42      0.08        40

avg / total       0.73      0.63      0.67      5244

request
             precision    recall  f1-score   support

          0       0.94      0.83      0.88      4347
          1       0.47      0.74      0.57       897

avg / total       0.86      0.81      0.83      5244

offer
             precision    recall  f1-score   support

          0       1.00      0.73      0.84      5221
          1       0.01      0.57      0.02        23

avg / total       0.99      0.73      0.84      5244

aid_related
             precision    recall  f1-score   support

          0       0.75      0.70      0.73      3082
          1       0.61      0.67      0.64      2162

avg / total       0.69      0.69      0.69      5244

medical_help
             precision    recall  f1-sco

In [69]:
#Param tunning 
new_parameters = {'clf__estimator__learning_rate': [1,2],
                'clf__estimator__n_estimators': [100, 200], 
                'tfidf__use_idf': (True, False)
            }

In [70]:
cv2 = GridSearchCV(new_pipeline2, param_grid=new_parameters,verbose=3)
cv2

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...er='best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__learning_rate': [1, 2], 'clf__estimator__n_estimators': [100, 200], 'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [71]:
cv2.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True, score=0.1171506222285796, total= 3.8min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.9min remaining:    0.0s


[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True, score=0.10813903590330425, total= 3.8min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  9.9min remaining:    0.0s


[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True, score=0.1061516452074392, total= 3.8min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=False 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=False, score=0.11400371906737233, total= 3.6min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=False 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=False, score=0.1085681590616507, total= 3.6min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=False 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=False, score=0.11015736766809728, total= 3.6min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=200, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=200, tfidf__use_

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 143.9min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...er='best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__learning_rate': [1, 2], 'clf__estimator__n_estimators': [100, 200], 'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [72]:
Report(cv2, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.43      0.78      0.55      1163
          1       0.92      0.70      0.79      4041
          2       0.38      0.38      0.38        40

avg / total       0.80      0.71      0.74      5244

request
             precision    recall  f1-score   support

          0       0.94      0.86      0.90      4347
          1       0.52      0.74      0.61       897

avg / total       0.87      0.84      0.85      5244

offer
             precision    recall  f1-score   support

          0       1.00      0.99      0.99      5221
          1       0.03      0.09      0.05        23

avg / total       0.99      0.99      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.81      0.80      0.81      3082
          1       0.72      0.73      0.73      2162

avg / total       0.77      0.77      0.77      5244

medical_help
             precision    recall  f1-sco

### 9. Export your model as a pickle file

In [75]:
with open('disaster_model.pkl', 'wb') as f:
    pickle.dump(cv2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.